# Project 2 report (for Shuxiao)

*Question: Are there any patterns between the number of bikes observed in roadpoints and their proximity to popular spots?*
Find t-test

#Shuxiao, to use the script in the Jupyter notebook follow the below code:#

In [1]:
import key_places_bikes as bikes
import pandas as pd

# this is the whole dataframe
hospital = bikes.HospitalBikes()
market = bikes.MarketBikes()

# select only pedal_cycles and then get a list. This will take about 3 minutes.. Be patient.
hospital_list = hospital["pedal_cycles"].tolist()
market_list = market["pedal_cycles"].tolist()

print(hospital_list)
print(market_list)

[36, 68, 36, 3, 1, 13, 0, 309, 101, 254, 1, 102, 147, 343, 39]
[149, 447, 85, 68, 42, 21, 1, 13, 17, 0, 1, 575, 613, 139, 0, 10, 56, 126, 1, 0, 133, 55]
